In [ ]:
import pandas as pd
import numpy as np
from zipline.api import order_target_percent, symbol, schedule_function, date_rules, time_rules, record
from zipline import run_algorithm
from zipline.data.data_portal import DataPortal
# from zipline.data.us_equity_pricing import BcolzDailyBarReader
from zipline.utils.calendars import get_calendar
from zipline.data.pandas_loader import PandasDataPortal
from zipline.assets import AssetFinder, Asset
from datetime import datetime

# === Load Your CSV ===
file_path = r'..\Data\OPCL_20000103_20201231.csv'
df = pd.read_csv(file_path)
df.set_index('ticker', inplace=True)
df.columns = pd.to_datetime(df.columns.str.lstrip('X'), format='%Y%m%d')
df = df.transpose()
df.index.name = 'date'
df = df.astype(float)
df.replace(0, np.nan, inplace=True)
df = df.ffill()

# === Format for Zipline-Reloaded: Make OHLCV Panel ===
assets = df.columns.tolist()
panel = {}
for asset in assets:
    prices = df[asset]
    panel[asset] = pd.DataFrame({
        'open': prices,
        'high': prices * 1.01,
        'low': prices * 0.99,
        'close': prices,
        'price': prices,
        'volume': 1e6
    })

# === Create asset metadata ===
from zipline.assets.synthetic import make_simple_equity_info
metadata = make_simple_equity_info(
    symbols=assets,
    start_date=df.index.min(),
    end_date=df.index.max(),
    exchange='NYSE'
)

# === Create the PandasDataPortal ===
data_portal = PandasDataPortal(
    equity_daily_bar=panel,
    asset_finder=AssetFinder(metadata=metadata),
    trading_calendar=get_calendar('XNYS')
)

# === Strategy Logic ===
def initialize(context):
    context.assets = [symbol(s) for s in assets]
    schedule_function(rebalance, date_rules.every_day(), time_rules.market_open())

def rebalance(context, data):
    weight = 1.0 / len(context.assets)
    for asset in context.assets:
        if data.can_trade(asset) and data.current(asset, 'price') > 1e-6:
            order_target_percent(asset, weight)
        else:
            order_target_percent(asset, 0)

def handle_data(context, data):
    record(**{asset.symbol: data.current(asset, 'price') for asset in context.assets[:3]})

# === Run the backtest ===
results = run_algorithm(
    start=df.index.min().tz_localize('UTC'),
    end=df.index.max().tz_localize('UTC'),
    initialize=initialize,
    handle_data=handle_data,
    capital_base=1e6,
    data_frequency='daily',
    data_portal=data_portal,
    trading_calendar=get_calendar('XNYS'),
)

print(results.tail())


ModuleNotFoundError: No module named 'zipline.data.us_equity_pricing'